<a href="https://colab.research.google.com/github/JennyFrost/ML_and_LLMs/blob/main/summarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install openai
!pip install python-docx

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 61.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 28.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 18.1 MB/s eta 0:00:00


In [ ]:
import openai
from docx import Document

In [ ]:
def extract_text_from_docx(doc_path):
    doc = Document(doc_path)
    text = " ".join([para.text for para in doc.paragraphs])
    return text

In [ ]:
file_name = '/content/Синхронизация Дубай-Маркетинг-20230605_153302-Запись собрания.docx'

In [ ]:
meeting_text = extract_text_from_docx(file_name)

$$GPT$$

In [ ]:
openai.api_key = ""

In [ ]:
system_message = "You are an office assistant."
user_message_base = """
                  Write a concise summary of this meeting minutes covering
                  main points and highlighting what needs to be done by whom.
                  Write in Russian.
               """ + '\n'

In [ ]:
def summarize(user_message, system_message=system_message, model="gpt-4-0314", temp=.7):
  response = openai.ChatCompletion.create(
          model=model,
          messages=[{"role": "system", "content": system_message},
                    {"role": "user", "content": user_message}],
          temperature=temp,
          top_p=1,
          frequency_penalty=0,
          presence_penalty=0
        )
  output_summary = response["choices"][0]["message"]["content"]
  return output_summary

In [ ]:
meeting_text_sents = meeting_text.split('.')
for n in range(2, 7):
    num_pieces = len(meeting_text_sents) // n
    pieces = []
    start = 0
    end = num_pieces
    print(f'Разбиение текста на {n} частей')
    print()
    for piece in range(n):
      text_piece = meeting_text_sents[start:end]
      start += num_pieces
      end += num_pieces
      user_message = user_message_base + ' '.join(text_piece)
      if piece == 0:
        user_message += "Also mention the purpose of the meeting." + '\n'
      if piece == n - 1:
        end = len(meeting_text_sents)
        user_message += "Also mention the outcome of the meeting." + '\n'
      try:
          print(f'Часть {piece+1}')
          print()
          user_message = user_message_base + ' '.join(text_piece)
          print(summarize(user_message=user_message))
          print()
      except openai.error.InvalidRequestError:
          print('Части слишком большие')
          print('=======================================================================================================================================')
          print()
          break
      print('===========================================================================================================================================')
      print()
